# * VINSIGHT : Churn Subs

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000200' --Prepaid Churn Subs
        , 'TB1S000200' --Prepaid Churn Subs : TMH
        , 'DB1S000200' --Prepaid Churn Subs : DTAC
        
        , 'B2S010200' --Postpaid Churn Subs B2C
        , 'TB2S010200' --Postpaid Churn Subs B2C : TMH
        , 'DB2S010200' --Postpaid Churn Subs B2C : DTAC
        , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
        , 'TB2S010201' --Postpaid Churn Subs Voluntary B2C : TMH
        , 'DB2S010201' --Postpaid Churn Subs Voluntary B2C : DTAC
        , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
        , 'TB2S010202' --Postpaid Churn Subs Involuntary B2C : TMH
        , 'DB2S010202' --Postpaid Churn Subs Involuntary B2C : DTAC
        , 'TB2S010210' --Postpaid Churn Subs B2C : TMH (Most Used Location)
        , 'TB2S010211' --Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
        , 'TB2S010212' --Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
        
        , 'B2S020200' --Postpaid Churn Subs B2B
        , 'TB2S020200' --Postpaid Churn Subs B2B : TMH
        , 'DB2S020200' --Postpaid Churn Subs B2B : DTAC
        , 'B2S020201' --Postpaid Churn Subs Voluntary B2B
        , 'TB2S020201' --Postpaid Churn Subs Voluntary B2B : TMH
        , 'DB2S020201' --Postpaid Churn Subs Voluntary B2B : DTAC
        , 'B2S020202' --Postpaid Churn Subs Involuntary B2B
        , 'TB2S020202' --Postpaid Churn Subs Involuntary B2B : TMH
        , 'DB2S020202' --Postpaid Churn Subs Involuntary B2B : DTAC
        , 'TB2S020210' --Postpaid Churn Subs B2B : TMH (Most Used Location)
        , 'TB2S020211' --Postpaid Churn Subs Voluntary B2B : TMH (Most Used Location)
        , 'TB2S020212' --Postpaid Churn Subs Involuntary B2B : TMH (Most Used Location)
        
        , 'TSER13100' --FTTx Churn Subs
        , 'TSER13102' --FTTx Churn Subs Voluntary
        , 'TSER13103' --FTTx Churn Subs Involuntary
        , 'TSER13104' --FTTx Churn Subs Involuntary (Collection Forecast)
        , 'TSER14100' --TVS Churn Subs
        , 'TSER14102' --TVS Churn Subs Voluntary
        , 'TSER14103' --TVS Churn Subs Involuntary
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-06-09, 12:18:18

DataFrame: 17816 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

## Daily

In [4]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202505


In [5]:
''' Churn Subs Daily '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    # , 'TB1S000200' #Prepaid Churn Subs : TMH
    # , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    # , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    # , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    # , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    # , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    # , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_sub_daily_df = churn_sub_daily_df.loc[churn_sub_daily_df['METRIC_CD'].isin(v_metric_list)]
churn_sub_daily_df = churn_sub_daily_df.loc[churn_sub_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_sub_daily_df['PRE'] = np.where(churn_sub_daily_df['METRIC_CD']=='B1S000200', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['FTTX'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13103', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['TVS'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14103', churn_sub_daily_df['P'], 0)

churn_sub_daily_df = churn_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_daily_df = churn_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
churn_sub_daily_df = churn_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]

mod_col_list = churn_sub_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_daily_df[col] = churn_sub_daily_df[col].apply(lambda x: format(x, ',.0f'))
churn_sub_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,202506,20250607,2025-06-08 14:08:34,0,0,0,0,0,0,0,0,0,0
1,202506,20250606,2025-06-08 14:08:34,"94,011",0,0,0,0,0,0,0,0,0
2,202506,20250605,2025-06-08 14:08:34,"108,556","11,518","8,673","3,870",0,0,0,905,76,0
3,202506,20250604,2025-06-08 14:08:34,"123,185",906,"2,953","3,102",0,0,0,885,76,0
4,202506,20250603,2025-06-08 14:08:34,"78,444","1,928","2,431",811,"10,115",607,1,"1,113",68,0
5,202506,20250602,2025-06-08 14:08:34,"76,806","14,668","2,364","6,794",562,562,0,"1,024",56,0
6,202506,20250601,2025-06-08 14:08:34,"67,669","4,299","2,960","2,275",-814,641,0,"1,142",146,0
7,202505,20250531,2025-06-08 14:08:34,"64,187","2,599","2,093","4,350",-355,600,31,"1,102",53,0
8,202505,20250530,2025-06-08 14:08:34,"20,154,633","4,794","2,593","3,667",-805,490,1,852,29,3
9,202505,20250529,2025-06-08 14:08:34,"73,586","15,380","2,527","7,363","-2,084",428,"3,077",83,5,0


In [6]:
''' Churn Subs(ALL, TRUE, DTAC) Daily '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_sub_daily_df = churn_sub_daily_df.loc[churn_sub_daily_df['METRIC_CD'].isin(v_metric_list)]
churn_sub_daily_df = churn_sub_daily_df.loc[churn_sub_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_sub_daily_df['PRE'] = np.where(churn_sub_daily_df['METRIC_CD']=='B1S000200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['PRE_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB1S000200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['PRE_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB1S000200', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['FTTX'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13103', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_IVL(CF)'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13104', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['TVS'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14103', churn_sub_daily_df['P'], 0)

churn_sub_daily_df = churn_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY'])\
    .agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
          , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
          , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
          , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
          , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
          , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_daily_df = churn_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
churn_sub_daily_df = churn_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
                                         , 'PRE', 'PRE_T', 'PRE_D'
                                         , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
                                         , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
                                         , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
                                         , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
                                         , 'TVS', 'TVS_VL', 'TVS_IVL']]

mod_col_list = churn_sub_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_daily_df[col] = churn_sub_daily_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{churn_sub_daily_df.to_string(max_cols=100)}') #max_rows=1000


    TM_KEY_MTH  TM_KEY_DAY              PPN_TM         PRE       PRE_T   PRE_D POST_B2C POST_B2C_VL POST_B2C_IVL POST_B2C_T POST_B2C_VL_T POST_B2C_IVL_T POST_B2C_D POST_B2C_VL_D POST_B2C_IVL_D    FTTX FTTX_VL FTTX_IVL    TVS TVS_VL TVS_IVL
0       202506    20250607 2025-06-08 14:08:34           0           0       0        0           0            0          0             0              0          0             0              0       0       0        0      0      0       0
1       202506    20250606 2025-06-08 14:08:34      94,011      66,338  27,673        0           0            0      5,163         2,186          2,977          0             0              0       0       0        0      0      0       0
2       202506    20250605 2025-06-08 14:08:34     108,556      76,714  31,842   11,518       8,673        3,870      9,142         5,608          3,534      2,376         3,065            336       0       0        0    905     76       0
3       202506    20250604 2025-06-08 1

## Products Summary

### Prep Monthly Data

In [7]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [8]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202506.0


### Prepaid

In [9]:
''' Prepaid '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


### Postpaid

In [10]:
''' Postpaid '''

v_product_grp = 'Postpaid'

postpaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[~postpaid_df['METRIC_NAME'].str.contains('B2C|B2B')]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


In [11]:
''' Postpaid B2C '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


In [12]:
''' Postpaid B2B '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


### TOL

In [13]:
''' TOL '''

v_product_grp = 'TOL'

tol_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


### TVS

In [14]:
''' TVS '''

v_product_grp = 'TVS'

tvs_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


## ** Current Issue

In [15]:
''' Issue : Prepaid Churn Subs : TMH (Align with BU) '''

v_metric_cd = 'TB1S000200'

issue_prepaid_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_prepaid_tmh_df = issue_prepaid_tmh_df.loc[issue_prepaid_tmh_df['TM_KEY_MTH']>=202401]
issue_prepaid_tmh_df = issue_prepaid_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_prepaid_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,411,781","1,217,672","1,201,070",0,"-1,411,781"
1,202402,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,481,911","1,276,487","1,260,774",0,"-1,481,911"
2,202403,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,445,434","1,292,312","1,276,558",0,"-1,445,434"
3,202404,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,556,142","1,402,113","1,384,897",0,"-1,556,142"
4,202405,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,420,875","-19,659,016","-19,674,697",0,"-1,420,875"
5,202406,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,119,897","-20,783,458","-20,783,458",0,"-1,119,897"
6,202407,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,606,946","-20,760,123","-20,760,123",0,"-1,606,946"
7,202408,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,142,088","-20,671,886","-20,671,886",0,"-1,142,088"
8,202409,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,376,492","-20,562,743","-20,562,743",0,"-1,376,492"
9,202410,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-08 12:02:51,0,"1,315,792","-20,429,168","-20,429,168",0,"-1,315,792"
